In [ ]:
# %pip install langchain chromadb pandas
# %pip install -qU langchain_community pypdf
!pip install tf-keras
!pip install groq




In [55]:
import chromadb
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
base_path = r"D:\summer 2_nd\Fake_yap"
file_name = "history.pdf"
name = "Fake_yap"
career = "music history"
model_list= ["compound-beta-mini","llama-3.3-70b-versatile"]

In [ ]:

loader = PyPDFLoader(f"{base_path}\{file_name}")
document = loader.load()
docs= [pages for pages in  document if len(pages.page_content.strip()) != 1 or 0 ] #deleting any null page if any
#-------------- creating chunks-------------------------------------
splitters = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap =40,
    separators=["\n\n", "\n", r"(?<=[.!?])\s", " ", ""]
)
chunks =[]
for pages in docs:
    chunks.append(pages.page_content)
print(chunks[0])
#------------------------Create embeddings--------------
embedder = SentenceTransformer("all-MiniLM-L12-v2") 
embeddings = embedder.encode(chunks, show_progress_bar=True)



In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path=r"D:\summer 2_nd\chroma_embeddings")

collection = client.get_or_create_collection(name=name)

collection.add(
    documents=chunks,
    embeddings= embeddings,
    ids = [f"page_{index}" for index in range(len(chunks))]

)

In [2]:
#getting the collection and embeddings from the chorma db 
client = chromadb.PersistentClient(path=r"D:\summer 2_nd\chroma_embeddings")
collection = client.get_collection(name=name)

In [29]:
#some questions for retrieval purposes 
questions = [
    # From the "In The Beginning" section
    "Why do you think early humans turned to music for rituals like weather prayers and hunting?",
    "What role do early homemade instruments, such as drums made from animal skins, play in our understanding of communal music making?",

    # From the "Middle Ages" section
    "How did naming musical notes and creating a writing system by Pope Gregory I around 600 A.D. transform the reach and power of music?",
    "What impact did Gregorian chant—a simple melody sung in unison—have on spiritual and communal experience?",
    "What drove the evolution from plainchant to elaborate masses involving choirs and orchestras?",

    # From the "Renaissance" section
    "How did the shift from monophonic chant to polyphonic madrigal reflect broader cultural and artistic developments?",
    "In what ways did religious patronage—like that of Palestrina by the Pope—influence Renaissance musical creativity?",
    "How did the madrigal lay the groundwork for the emergence of opera during the Renaissance?",
    "What innovations did Monteverdi bring by combining memorable melodies, stage storytelling, and instrumental accompaniment?",
    "Why do you think opera became the dominant form of musical entertainment in the late Renaissance?",

    # From the "Baroque Era" section
    "Baroque music is known for its ornamentation and expressive melodies—what do you think this says about the era’s artistic values?",
    "Vivaldi’s 'Four Seasons' paints musical pictures of nature—how do you think music can convey feelings or scenes without any words?",
    "Handel transitioned from opera to oratorio—how did using biblical texts and orchestras help connect with larger audiences?",
    "Why do you think Handel’s 'Messiah' remains one of the most enduring oratorios in music history?",
    "Bach was known for improvising on the organ and composing complex fugues—how does his approach show the balance between structure and creativity?",
    "How might growing up with 20 children copying music in the Bach household have influenced his productivity and legacy?",
    "What makes a fugue—with multiple melodies layered at different times—so uniquely challenging and impressive to perform?",

    # From the "Classical Era" section
    "Why do you think the Classical Era moved toward simpler, more structured music compared to the complex Baroque style?",
    "Joseph Haydn is known as the father of the symphony—how did his innovations shape the future of orchestral music?",
    "What role did Vienna and Austria play in shaping Classical music through composers like Haydn, Mozart, and Beethoven?",
    "How did the values of the Classical era—clarity, balance, and form—reflect broader Enlightenment thinking?"
]


In [38]:
some_query = questions[:3]
print(some_query)
results = collection.query(
    query_texts= some_query, 
    n_results=2,
    include=["documents", "embeddings", "distances"]
)

['Why do you think early humans turned to music for rituals like weather prayers and hunting?', 'What role do early homemade instruments, such as drums made from animal skins, play in our understanding of communal music making?', 'How did naming musical notes and creating a writing system by Pope Gregory I around 600 A.D. transform the reach and power of music?']


In [44]:
career = "music history"
guest_name ="wobily wiggly"

persona_prompt_podcaster= "you are a famous and friendly youtube podcaster who is asking questions from your guest whos name is {guest_name}   and dont give visual cues and just ask 1 question a time "
persona_prompt_guest = f"you are a friendly {career} professional answering questions from a famous podcaster, having a friendly podcast convesation  and dont give visual cues"
welcome_prompt = f"Welcome the guess to your podcast who is a {career} professional."
thanks_prompt = f"Thanks the podcsater for having you on the show"
bye_prompt_podcaster = f"Say thankyou and good bye to the {career} professional who came on your show"
bye_prompt_guest = f"Say thank you and good bye to the podcaster who had you on the show"
def ask_q(question):
    question_asking_baseline = f"""ask the question from the guest and repeat it inyour words
    example : Let me ask you what is the meaning of RAG?
    example2:  so can you tell me about dinosours

    your question is : {questions}
    """
    return question_asking_baseline

In [45]:
import os
import dotenv
from dotenv import load_dotenv
groq_api = os.getenv("groq_api")


In [58]:
from groq import Groq
client = Groq(api_key=groq_api)
history = []  # Properly formatted history as a list of message dicts

def podcaster_speaks(history, prompt):
    global persona_prompt_podcaster

    # Wrap the new user message
    messages = [{"role": "system", "content": persona_prompt_podcaster}] + history + [
        {"role": "user", "content": prompt}
    ]

    podcaster = client.chat.completions.create(
        model=model_list[0],
        messages=messages,
        temperature=0.5,
        max_completion_tokens=300,
        top_p=1,
    )

    reply = podcaster.choices[0].message
    history.append({"role": reply.role, "content": reply.content})
    return reply.content

def podcaster_questions(history, prompt):
    global persona_prompt_podcaster
    question = ask_q(prompt)
    messages = [{"role": "system", "content": persona_prompt_podcaster}] + history + [
        {"role": "user", "content": question}
    ]

    podcaster = client.chat.completions.create(
        model=model_list[0],
        messages=messages,
        temperature=0.5,
        max_completion_tokens=300,
        top_p=1,
    )

    reply = podcaster.choices[0].message
    history.append({"role": reply.role, "content": reply.content})
    return reply.content

def guest_speak(history, prompt):
    global persona_prompt_guest

    # Step 1: Get the most relevant chunk from ChromaDB
    results = collection.query(
        query_texts=[prompt],
        n_results=1,
        include=["documents"]
    )

    context = results['documents'][0][0] if results['documents'][0] else "No relevant document found."

    # Step 2: Set system prompt to instruct model to use only the context
    strict_system_prompt = f"""{persona_prompt_guest}

Only answer using the following document. Do not make up or infer any information that is not explicitly stated but you can change the wording and make it look like you are saying it 
---
{context}
---
"""

    # Step 3: Combine with chat history
    messages = [{"role": "system", "content": strict_system_prompt}] + history + [
        {"role": "user", "content": prompt}
    ]

    # Step 4: Call the model
    guest = client.chat.completions.create(
        model=model_list[0],
        messages=messages,
        temperature=0.0,  # Reduce creativity for stricter grounding
        max_completion_tokens=300,
        top_p=1
    )

    # Step 5: Update history and return reply
    reply = guest.choices[0].message
    history.append({"role": reply.role, "content": reply.content})
    return reply.content


In [59]:
welcome = False
questions_count = 0
question_index = 0
all_questions = questions[:4]  # your question list

while True:
    if not welcome:
        print("----------- Podcaster Turn -----------\n")
        p = podcaster_speaks(history, welcome_prompt)
        print(p)

        print("\n----------- Guest Turn -----------\n")
        q = guest_speak(history, thanks_prompt)
        print(q)

        welcome = True

    elif questions_count < len(all_questions):
        print("\n----------- Podcaster Turn -----------\n")
        p = podcaster_questions(history, all_questions[question_index])
        print(p)

        print("\n----------- Guest Turn -----------\n")
        q = guest_speak(history, all_questions[question_index])
        print(q)

        question_index += 1
        questions_count += 1

    else:
        print("\n----------- Podcaster Turn -----------\n")
        p = podcaster_speaks(history, bye_prompt_podcaster)
        print(p)

        print("\n----------- Guest Turn -----------\n")
        q = guest_speak(history, bye_prompt_guest)
        print(q)
        break


----------- Podcaster Turn -----------

What's up everyone, welcome back to our channel, today I'm super excited to have with me, Dr. Rachel Kim, a renowned music history professional, Rachel thanks for being on the show! 

Can you tell us, what sparked your interest in music history, and how did you become an expert in this field?

----------- Guest Turn -----------



RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01k0exqfqpf3nsha8v8p5n45jc` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99811, Requested 1050. Please try again in 12m23.366s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'compound', 'code': 'rate_limit_exceeded'}}